In [1]:
#standard ds imports
import pandas as pd
import numpy as np
#viz and stats
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
# .py imports
from env import user, pwd,host
import env
import wranglerer as wr
#import modeling as md
import os
#sklearn imports
from sklearn.model_selection import train_test_split
import sklearn.preprocessing
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import ListedColormap
from sklearn.metrics import mean_squared_error,explained_variance_score

from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.feature_selection import SelectKBest, RFE, f_regression, SequentialFeatureSelector
from sklearn.metrics import r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
#CATboost imports
from catboost import CatBoostRegressor

## ACQUIRE

In [2]:
# ACQUIRE/PREP
df = wr.merge_wines_df()

local version found!
local version found!


## SPLIT

In [3]:
# SPLIT
X_train, y_train, X_validate, y_validate, X_test, y_test = wr.train_validate_test(df,'quality')

In [4]:
X_train.shape, y_train.shape, X_validate.shape, y_validate.shape, X_test.shape, y_test.shape

((3637, 12), (3637,), (1560, 12), (1560,), (1300, 12), (1300,))

In [5]:
X_train.columns

Index(['fixed_acidity', 'volatile_acidity', 'citric_acid', 'residual_sugar',
       'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'density',
       'ph', 'sulphates', 'alcohol', 'wine_color'],
      dtype='object')

## SCALE (MinMax)

In [6]:
# SCALE THE NUMERICAL DRIVERS
cols = X_train.columns
X_train_scaled, X_validate_scaled, X_test_scaled = wr.min_max_scale(X_train,X_validate,X_test,cols)

## CLUSTER (f)

In [7]:
X_train_scaled.columns

Index(['fixed_acidity', 'volatile_acidity', 'citric_acid', 'residual_sugar',
       'chlorides', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'density',
       'ph', 'sulphates', 'alcohol', 'wine_color'],
      dtype='object')

In [8]:
#define features for CLUSTERING:
clustering_feats = ['total_sulfur_dioxide',
                'volatile_acidity',
                'chlorides']
X_train_cl = X_train[clustering_feats]
X_validate_cl = X_validate[clustering_feats]
X_test_cl = X_test[clustering_feats]

In [9]:
# lets make a clustering object from sklearn
# Make a thing! Thats my favorite!
k_means_proto = KMeans(n_clusters=2)
# fit the thing!!!!
k_means_proto.fit(X_train_cl)
# use the thing
X_train_clusters = k_means_proto.predict(X_train_cl)

In [10]:
X_train_cl.head()

,total_sulfur_dioxide,volatile_acidity,chlorides
3737,102.0,0.24,0.060
19,56.0,0.32,0.341
2200,106.0,0.24,0.045
4139,177.0,0.29,0.030
501,76.0,0.44,0.074


In [11]:
X_train_color = X_train.wine_color

In [12]:
X_train_color

3737    0
19      1
2200    0
4139    0
501     1
       ..
4918    0
3542    0
1884    0
2206    0
548     1
Name: wine_color, Length: 3637, dtype: int64

In [13]:
X_train_clusters

array([1, 1, 1, ..., 0, 0, 1], dtype=int32)

In [14]:
X_train_cl_results = pd.DataFrame({'actual_color': X_train_color,
                                  'clust_color': X_train_clusters})

In [15]:
X_train_cl_results

,actual_color,clust_color
3737,0,1
19,1,1
2200,0,1
4139,0,0
501,1,1
...,...,...
4918,0,0
3542,0,0
1884,0,0
2206,0,0


In [16]:
X_clusters = pd.concat([X_train_cl,X_train_cl_results],axis=1)

In [17]:
X_clusters

,total_sulfur_dioxide,volatile_acidity,chlorides,actual_color,clust_color
3737,102.0,0.24,0.060,0,1
19,56.0,0.32,0.341,1,1
2200,106.0,0.24,0.045,0,1
4139,177.0,0.29,0.030,0,0
501,76.0,0.44,0.074,1,1
...,...,...,...,...,...
4918,148.0,0.21,0.047,0,0
3542,195.0,0.25,0.041,0,0
1884,180.0,0.32,0.056,0,0
2206,158.0,0.25,0.049,0,0


In [18]:
X_clusters['success'] = X_clusters.actual_color == X_clusters.clust_color

In [19]:
X_clusters

,total_sulfur_dioxide,volatile_acidity,chlorides,actual_color,clust_color,success
3737,102.0,0.24,0.060,0,1,False
19,56.0,0.32,0.341,1,1,True
2200,106.0,0.24,0.045,0,1,False
4139,177.0,0.29,0.030,0,0,True
501,76.0,0.44,0.074,1,1,True
...,...,...,...,...,...,...
4918,148.0,0.21,0.047,0,0,True
3542,195.0,0.25,0.041,0,0,True
1884,180.0,0.32,0.056,0,0,True
2206,158.0,0.25,0.049,0,0,True


In [20]:
X_clusters.success.value_counts()

True     2890
False     747
Name: success, dtype: int64

In [21]:
tr = X_clusters.success.value_counts()[1]
fal = X_clusters.success.value_counts()[0]

round(tr / (tr+fal),3)

0.795

## Running a K-means with 2 centroids as a way to potentially replicate the two-color split.  Vanilla K-means was able to identify 79.5% of actual color-values.

## However, wine_color is NOT our target, so this information would only be useful should we need to subset our data based upon wine_color.

## CLUSTERING 2

In [22]:
#define features for CLUSTERING:
clustering_feats = ['total_sulfur_dioxide',
                'volatile_acidity']
X_train_cl = X_train[clustering_feats]
X_validate_cl = X_validate[clustering_feats]
X_test_cl = X_test[clustering_feats]

In [23]:
# lets make a clustering object from sklearn
# Make a thing! Thats my favorite!
k_means_proto = KMeans(n_clusters=2)
# fit the thing!!!!
k_means_proto.fit(X_train_cl)
# use the thing
X_train_clusters = k_means_proto.predict(X_train_cl)


In [24]:
X_train_cl_results = pd.DataFrame({'actual_color2': X_train_color,'clust_color2': X_train_clusters})

In [25]:
X_train_cl_results.head()

,actual_color2,clust_color2
3737,0,0
19,1,0
2200,0,0
4139,0,1
501,1,0


In [26]:
X_clusters = pd.concat([X_clusters,X_train_cl_results],axis=1)
X_clusters['success2'] = X_clusters.actual_color2 == X_clusters.clust_color2

In [27]:
X_clusters

,total_sulfur_dioxide,volatile_acidity,chlorides,actual_color,clust_color,success,actual_color2,clust_color2,success2
3737,102.0,0.24,0.060,0,1,False,0,0,True
19,56.0,0.32,0.341,1,1,True,1,0,False
2200,106.0,0.24,0.045,0,1,False,0,0,True
4139,177.0,0.29,0.030,0,0,True,0,1,False
501,76.0,0.44,0.074,1,1,True,1,0,False
...,...,...,...,...,...,...,...,...,...
4918,148.0,0.21,0.047,0,0,True,0,1,False
3542,195.0,0.25,0.041,0,0,True,0,1,False
1884,180.0,0.32,0.056,0,0,True,0,1,False
2206,158.0,0.25,0.049,0,0,True,0,1,False


In [28]:
tr = X_clusters.success2.value_counts()[1]
fal = X_clusters.success2.value_counts()[0]

round(tr / (tr+fal),3)

0.205

## CLUSTERING 3

In [29]:
#define features for CLUSTERING:
clustering_feats = ['chlorides',
                'volatile_acidity']
X_train_cl = X_train[clustering_feats]
X_validate_cl = X_validate[clustering_feats]
X_test_cl = X_test[clustering_feats]

# lets make a clustering object from sklearn
# Make a thing! Thats my favorite!
k_means_proto = KMeans(n_clusters=2)
# fit the thing!!!!
k_means_proto.fit(X_train_cl)
# use the thing
X_train_clusters = k_means_proto.predict(X_train_cl)

X_train_cl_results = pd.DataFrame({'actual_color3': X_train_color,'clust_color3': X_train_clusters})

X_clusters = pd.concat([X_clusters,X_train_cl_results],axis=1)
X_clusters['success3'] = X_clusters.actual_color3 == X_clusters.clust_color3

tr = X_clusters.success3.value_counts()[1]
fal = X_clusters.success3.value_counts()[0]

round(tr / (tr+fal),3)

0.868

In [30]:
X_clusters.head()

,total_sulfur_dioxide,volatile_acidity,chlorides,actual_color,clust_color,success,actual_color2,clust_color2,success2,actual_color3,clust_color3,success3
3737,102.0,0.24,0.060,0,1,False,0,0,True,0,0,True
19,56.0,0.32,0.341,1,1,True,1,0,False,1,0,False
2200,106.0,0.24,0.045,0,1,False,0,0,True,0,0,True
4139,177.0,0.29,0.030,0,0,True,0,1,False,0,0,True
501,76.0,0.44,0.074,1,1,True,1,0,False,1,1,True


## EST BASELINE (mean)

In [31]:
# ESTABLISH BASELINE:
# convert y_targets into dataframe so that we can append additional columns to it
y_train = pd.DataFrame(y_train)
y_validate = pd.DataFrame(y_validate)
y_test = pd.DataFrame(y_test)

# Predict target_pred_mean-Baseline
target_pred_mean = y_train.quality.mean()
y_train['target_pred_mean'] = round(target_pred_mean,0)
y_validate['target_pred_mean'] = target_pred_mean

# 2. compute target_pred_median
target_pred_median = y_train.quality.median()
y_train['target_pred_median'] = target_pred_median
y_validate['target_pred_median'] = target_pred_median

#Calculate RMSE for Baseline:
rmse_train_mean = round(mean_squared_error(y_train.quality,
                                y_train.target_pred_mean) ** (.5),1)
rmse_validate_mean = round(mean_squared_error(y_validate.quality, y_validate.target_pred_mean) ** (0.5),1)

rmse_train_med = mean_squared_error(y_train.quality, y_train.target_pred_median) ** .5
rmse_validate_med = mean_squared_error(y_validate.quality, y_validate.target_pred_median) ** (0.5)

print(f"""RMSE using Mean
Train/In-Sample: {round(rmse_train_mean, 1)} 
Validate/Out-of-Sample: {round(rmse_validate_mean, 1)}""")
print(f"""RMSE using Median
Train/In-Sample: {round(rmse_train_med, 2)} 
Validate/Out-of-Sample: {round(rmse_validate_med, 2)}""")

# USE MEAN as BASELINE

RMSE using Mean
Train/In-Sample: 0.9 
Validate/Out-of-Sample: 0.9
RMSE using Median
Train/In-Sample: 0.89 
Validate/Out-of-Sample: 0.91


In [32]:
# Drop Median...remove unnecessary clutter
y_train = y_train.drop(columns='target_pred_median')

## K-BEST / RFE

In [33]:
# K-BEST results:
# make the thing
kbest = SelectKBest(f_regression,k=4)
# fit the thing
_ = kbest.fit(X_train_scaled,y_train.quality)
# statistical f-value:
kbest.scores_
X_train_scaled.columns[kbest.get_support()]

Index(['volatile_acidity', 'chlorides', 'density', 'alcohol'], dtype='object')

In [34]:
y_train

,quality,target_pred_mean
3737,5,6.0
19,6,6.0
2200,6,6.0
4139,5,6.0
501,7,6.0
...,...,...
4918,5,6.0
3542,5,6.0
1884,5,6.0
2206,5,6.0


In [35]:
# RFE results:
# establish a model for RFE to use
model = LinearRegression()
# make an RFE thing
rfe = RFE(model, n_features_to_select=4)
# fit the RFE thing
rfe.fit(X_train_scaled,y_train.quality)
rfe_df = pd.DataFrame({
    'rfe_ranking':rfe.ranking_
}, index=X_train_scaled.columns)
rfe_df.sort_values('rfe_ranking')

,rfe_ranking
volatile_acidity,1
residual_sugar,1
sulphates,1
alcohol,1
free_sulfur_dioxide,2
total_sulfur_dioxide,3
density,4
fixed_acidity,5
ph,6
chlorides,7


## METRIC_DF

In [36]:
# build a quick df to compare results...will append to this df as needed
metric_df = pd.DataFrame(data=[
    {'model': 'mean_baseline',
     'RMSE_train': rmse_train_mean,
     'RMSE_validate': rmse_validate_mean,
     'R2_validate': explained_variance_score(y_validate.quality,
                                             y_validate.target_pred_mean)
    }
]
)
metric_df

,model,RMSE_train,RMSE_validate,R2_validate
0,mean_baseline,0.9,0.9,0.0


## MODEL 1 OLS
- volatile_acidity, alcohol, density

In [37]:
#define features for MODEL 1:
model_1_feats = ['alcohol',
                'volatile_acidity',
                'density']
X_train_1 = X_train[model_1_feats]
X_validate_1 = X_validate[model_1_feats]
X_test_1 = X_test[model_1_feats]

# MAKE THE THING: create the model object
lm = LinearRegression()
# FIT THE THING: fit the model to training data
OLSmodel = lm.fit(X_train_1, y_train.quality)
# USE THE THING: make a prediction
y_train['target_pred_lm'] = lm.predict(X_train_1)
#Evaluate: RMSE
rmse_train = mean_squared_error(y_train.quality, y_train.target_pred_lm)**(1/2)
# predict validate
y_validate['target_pred_lm'] = lm.predict(X_validate_1)
# evaluate: RMSE
rmse_validate = mean_squared_error(y_validate.quality, y_validate.target_pred_lm)**(1/2)
print(f"""RMSE for OLS using LinearRegression
Training/In-Sample:  {rmse_train} 
Validation/Out-of-Sample: {rmse_validate}""")

RMSE for OLS using LinearRegression
Training/In-Sample:  0.7436609095570793 
Validation/Out-of-Sample: 0.7557847149398661


In [38]:
#Append this to the metric_df
metric_df = metric_df.append(
    {'model': 'OLS Model 1 (base feats)',
     'RMSE_train': round(rmse_train,2),
     'RMSE_validate': round(rmse_validate,2),
     'R2_validate': round(explained_variance_score(y_validate.quality,
                                             y_validate.target_pred_lm),4)
    }, ignore_index=True)
metric_df

/var/folders/ty/2pnn6wsj06n_glzjgbmxzlkw0000gn/T/ipykernel_2015/1027816449.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metric_df = metric_df.append(


,model,RMSE_train,RMSE_validate,R2_validate
0,mean_baseline,0.90,0.90,0.0000
1,OLS Model 1 (base feats),0.74,0.76,0.2782


## MODEL 1 OLS
- add wine_color

In [39]:
#define features for MODEL 1:
model_1_feats = ['alcohol',
                'volatile_acidity',
                'density',
                'wine_color']
X_train_1 = X_train[model_1_feats]
X_validate_1 = X_validate[model_1_feats]
X_test_1 = X_test[model_1_feats]

# MAKE THE THING: create the model object
lm = LinearRegression()
# FIT THE THING: fit the model to training data
OLSmodel = lm.fit(X_train_1, y_train.quality)
# USE THE THING: make a prediction
y_train['target_pred_lm'] = lm.predict(X_train_1)
#Evaluate: RMSE
rmse_train = mean_squared_error(y_train.quality, y_train.target_pred_lm)**(1/2)
# predict validate
y_validate['target_pred_lm'] = lm.predict(X_validate_1)
# evaluate: RMSE
rmse_validate = mean_squared_error(y_validate.quality, y_validate.target_pred_lm)**(1/2)
print(f"""RMSE for OLS using LinearRegression
Training/In-Sample:  {rmse_train} 
Validation/Out-of-Sample: {rmse_validate}""")

RMSE for OLS using LinearRegression
Training/In-Sample:  0.7431262818804543 
Validation/Out-of-Sample: 0.7536053512471359


In [40]:
#Append this to the metric_df
metric_df = metric_df.append(
    {'model': 'OLS Model 1 (base + color)',
     'RMSE_train': round(rmse_train,2),
     'RMSE_validate': round(rmse_validate,2),
     'R2_validate': round(explained_variance_score(y_validate.quality,
                                             y_validate.target_pred_lm),4)
    }, ignore_index=True)
metric_df

/var/folders/ty/2pnn6wsj06n_glzjgbmxzlkw0000gn/T/ipykernel_2015/1224112532.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metric_df = metric_df.append(


,model,RMSE_train,RMSE_validate,R2_validate
0,mean_baseline,0.90,0.90,0.0000
1,OLS Model 1 (base feats),0.74,0.76,0.2782
2,OLS Model 1 (base + color),0.74,0.75,0.2823


## MODEL 1 OLS
- add chlorides

In [41]:
#define features for MODEL 1:
model_1_feats = ['alcohol',
                'volatile_acidity',
                'density',
                'wine_color',
                'chlorides']
X_train_1 = X_train[model_1_feats]
X_validate_1 = X_validate[model_1_feats]
X_test_1 = X_test[model_1_feats]

# MAKE THE THING: create the model object
lm = LinearRegression()
# FIT THE THING: fit the model to training data
OLSmodel = lm.fit(X_train_1, y_train.quality)
# USE THE THING: make a prediction
y_train['target_pred_lm'] = lm.predict(X_train_1)
#Evaluate: RMSE
rmse_train = mean_squared_error(y_train.quality, y_train.target_pred_lm)**(1/2)
# predict validate
y_validate['target_pred_lm'] = lm.predict(X_validate_1)
# evaluate: RMSE
rmse_validate = mean_squared_error(y_validate.quality, y_validate.target_pred_lm)**(1/2)
print(f"""RMSE for OLS using LinearRegression
Training/In-Sample:  {rmse_train} 
Validation/Out-of-Sample: {rmse_validate}""")

RMSE for OLS using LinearRegression
Training/In-Sample:  0.7425498029952226 
Validation/Out-of-Sample: 0.7529423946142223


In [42]:
#Append this to the metric_df
metric_df = metric_df.append(
    {'model': 'OLS Model 1 (+ chlorides)',
     'RMSE_train': round(rmse_train,2),
     'RMSE_validate': round(rmse_validate,2),
     'R2_validate': round(explained_variance_score(y_validate.quality,
                                             y_validate.target_pred_lm),4)
    }, ignore_index=True)
metric_df

/var/folders/ty/2pnn6wsj06n_glzjgbmxzlkw0000gn/T/ipykernel_2015/858112617.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metric_df = metric_df.append(


,model,RMSE_train,RMSE_validate,R2_validate
0,mean_baseline,0.90,0.90,0.0000
1,OLS Model 1 (base feats),0.74,0.76,0.2782
2,OLS Model 1 (base + color),0.74,0.75,0.2823
3,OLS Model 1 (+ chlorides),0.74,0.75,0.2836


## MODEL 1 OLS
- add sulphates

In [43]:
#define features for MODEL 1:
model_1_feats = ['alcohol',
                'volatile_acidity',
                'density',
                'wine_color',
                'chlorides',
                'sulphates']
X_train_1 = X_train[model_1_feats]
X_validate_1 = X_validate[model_1_feats]
X_test_1 = X_test[model_1_feats]

# MAKE THE THING: create the model object
lm = LinearRegression()
# FIT THE THING: fit the model to training data
OLSmodel = lm.fit(X_train_1, y_train.quality)
# USE THE THING: make a prediction
y_train['target_pred_lm'] = lm.predict(X_train_1)
#Evaluate: RMSE
rmse_train = mean_squared_error(y_train.quality, y_train.target_pred_lm)**(1/2)
# predict validate
y_validate['target_pred_lm'] = lm.predict(X_validate_1)
# evaluate: RMSE
rmse_validate = mean_squared_error(y_validate.quality, y_validate.target_pred_lm)**(1/2)
print(f"""RMSE for OLS using LinearRegression
Training/In-Sample:  {rmse_train} 
Validation/Out-of-Sample: {rmse_validate}""")

RMSE for OLS using LinearRegression
Training/In-Sample:  0.7401442039707741 
Validation/Out-of-Sample: 0.7483438899735064


In [44]:
#Append this to the metric_df
metric_df = metric_df.append(
    {'model': 'OLS Model 1 (+ sulphates)',
     'RMSE_train': round(rmse_train,2),
     'RMSE_validate': round(rmse_validate,2),
     'R2_validate': round(explained_variance_score(y_validate.quality,
                                             y_validate.target_pred_lm),4)
    }, ignore_index=True)
metric_df

/var/folders/ty/2pnn6wsj06n_glzjgbmxzlkw0000gn/T/ipykernel_2015/1561224617.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metric_df = metric_df.append(


,model,RMSE_train,RMSE_validate,R2_validate
0,mean_baseline,0.90,0.90,0.0000
1,OLS Model 1 (base feats),0.74,0.76,0.2782
2,OLS Model 1 (base + color),0.74,0.75,0.2823
3,OLS Model 1 (+ chlorides),0.74,0.75,0.2836
4,OLS Model 1 (+ sulphates),0.74,0.75,0.2923


In [45]:
#define features for MODEL 1:
model_1_feats = ['alcohol',
                'volatile_acidity',
                'density',
                'wine_color',
                'chlorides',
                'sulphates']
X_train_1 = X_train_scaled[model_1_feats]
X_validate_1 = X_validate_scaled[model_1_feats]
X_test_1 = X_test_scaled[model_1_feats]

# MAKE THE THING: create the model object
lm = LinearRegression()
# FIT THE THING: fit the model to training data
OLSmodel = lm.fit(X_train_1, y_train.quality)
# USE THE THING: make a prediction
y_train['target_pred_lm'] = lm.predict(X_train_1)
#Evaluate: RMSE
rmse_train = mean_squared_error(y_train.quality, y_train.target_pred_lm)**(1/2)
# predict validate
y_validate['target_pred_lm'] = lm.predict(X_validate_1)
# evaluate: RMSE
rmse_validate = mean_squared_error(y_validate.quality, y_validate.target_pred_lm)**(1/2)
print(f"""RMSE for OLS using LinearRegression
Training/In-Sample:  {rmse_train} 
Validation/Out-of-Sample: {rmse_validate}""")

RMSE for OLS using LinearRegression
Training/In-Sample:  0.7401442039707741 
Validation/Out-of-Sample: 0.7483438899735064


In [46]:
#Append this to the metric_df
metric_df = metric_df.append(
    {'model': 'OLS Model 1 Scaled (+ sulphates)',
     'RMSE_train': round(rmse_train,2),
     'RMSE_validate': round(rmse_validate,2),
     'R2_validate': round(explained_variance_score(y_validate.quality,
                                             y_validate.target_pred_lm),4)
    }, ignore_index=True)
metric_df

/var/folders/ty/2pnn6wsj06n_glzjgbmxzlkw0000gn/T/ipykernel_2015/178744154.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metric_df = metric_df.append(


,model,RMSE_train,RMSE_validate,R2_validate
0,mean_baseline,0.90,0.90,0.0000
1,OLS Model 1 (base feats),0.74,0.76,0.2782
2,OLS Model 1 (base + color),0.74,0.75,0.2823
3,OLS Model 1 (+ chlorides),0.74,0.75,0.2836
4,OLS Model 1 (+ sulphates),0.74,0.75,0.2923
5,OLS Model 1 Scaled (+ sulphates),0.74,0.75,0.2923


## MODEL 2 DTR maxDepth=7
- volatile_acidity, alcohol, density

In [47]:
model_2_feats = ['alcohol',
                'volatile_acidity',
                'density',
                'wine_color',
                'chlorides',
                'sulphates',
                'citric_acid']
X_train_1 = X_train[model_2_feats]
X_validate_1 = X_validate[model_2_feats]
X_test_1 = X_test[model_2_feats]

# Create and fit the thing
DTRp = DecisionTreeRegressor(max_depth=5)
DTRp.fit(X_train_1,y_train.quality)
y_train['target_pred_DTRp'] = DTRp.predict(X_train_1)
DTRp.predict(X_train_1)
# evaluate: RMSE
rmse_train = mean_squared_error(y_train.quality, y_train.target_pred_DTRp)**(1/2)
# predict validate
y_validate['target_pred_DTRp'] = DTRp.predict(X_validate_1)
# evaluate: RMSE
rmse_validate = mean_squared_error(y_validate.quality, y_validate.target_pred_DTRp)**(1/2)
r2 = r2_score(y_train.quality,y_train.target_pred_DTRp)
print(f"""RMSE for OLS using LinearRegression
Training/In-Sample:  {rmse_train} 
Validation/Out-of-Sample: {rmse_validate}
Train R2 Score: {r2}""")

RMSE for OLS using LinearRegression
Training/In-Sample:  0.700572389727326 
Validation/Out-of-Sample: 0.7480412396615373
Train R2 Score: 0.34597177670463775


In [48]:
#Append this to the metric_df
metric_df = metric_df.append(
    {'model': 'DTR: maxD=5 (+ citric)',
     'RMSE_train': round(rmse_train,2),
     'RMSE_validate': round(rmse_validate,2),
     'R2_validate': round(explained_variance_score(y_validate.quality,
                                             y_validate.target_pred_DTRp),4)
    }, ignore_index=True)
metric_df

/var/folders/ty/2pnn6wsj06n_glzjgbmxzlkw0000gn/T/ipykernel_2015/2999740504.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metric_df = metric_df.append(


,model,RMSE_train,RMSE_validate,R2_validate
0,mean_baseline,0.90,0.90,0.0000
1,OLS Model 1 (base feats),0.74,0.76,0.2782
2,OLS Model 1 (base + color),0.74,0.75,0.2823
3,OLS Model 1 (+ chlorides),0.74,0.75,0.2836
4,OLS Model 1 (+ sulphates),0.74,0.75,0.2923
5,OLS Model 1 Scaled (+ sulphates),0.74,0.75,0.2923
6,DTR: maxD=5 (+ citric),0.70,0.75,0.2932


## MODEL 3 RFR n_estimators=55, maxDepth= 5
- volatile_acidity, alcohol, density

In [49]:
#model_3_feats = ['alcohol',
#                 'volatile_acidity',
#                 'density',
#                 'wine_color',
#                 'chlorides',
#                 'sulphates',
#                 'citric_acid']
X_train_1 = X_train
X_validate_1 = X_validate
X_test_1 = X_test

# Create and fit the thing
RFRp = RandomForestRegressor(n_estimators=301, max_depth=10,random_state=2013)
RFRp.fit(X_train_1,y_train.quality)
y_train['target_pred_RFRp'] = RFRp.predict(X_train_1)
RFRp.predict(X_train_1)
#Evaluate: RMSE
rmse_train = mean_squared_error(y_train.quality, y_train.target_pred_RFRp)**(1/2)
#REPEAT STEPS 2-3
# predict validate
y_validate['target_pred_RFRp'] = RFRp.predict(X_validate_1)
# evaluate: RMSE
rmse_validate = mean_squared_error(y_validate.quality, y_validate.target_pred_RFRp)**(1/2)
r2 = r2_score(y_train.quality,y_train.target_pred_RFRp)
print(f"""RMSE for OLS using LinearRegression
Training/In-Sample:  {rmse_train} 
Validation/Out-of-Sample: {rmse_validate}
Train R2 Score: {r2}""")
#Append this to the metric_df
metric_df = metric_df.append(
    {'model': 'RFR ALL FEATS est=301, maxD=10',
     'RMSE_train': round(rmse_train,2),
     'RMSE_validate': round(rmse_validate,2),
     'R2_validate': round(explained_variance_score(y_validate.quality,
                                             y_validate.target_pred_RFRp),4)
    }, ignore_index=True)

metric_df

RMSE for OLS using LinearRegression
Training/In-Sample:  0.45439360534061496 
Validation/Out-of-Sample: 0.6559099717886514
Train R2 Score: 0.7248592751571042


/var/folders/ty/2pnn6wsj06n_glzjgbmxzlkw0000gn/T/ipykernel_2015/2656577834.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metric_df = metric_df.append(


,model,RMSE_train,RMSE_validate,R2_validate
0,mean_baseline,0.90,0.90,0.0000
1,OLS Model 1 (base feats),0.74,0.76,0.2782
2,OLS Model 1 (base + color),0.74,0.75,0.2823
3,OLS Model 1 (+ chlorides),0.74,0.75,0.2836
4,OLS Model 1 (+ sulphates),0.74,0.75,0.2923
5,OLS Model 1 Scaled (+ sulphates),0.74,0.75,0.2923
6,DTR: maxD=5 (+ citric),0.70,0.75,0.2932
7,"RFR ALL FEATS est=301, maxD=10",0.45,0.66,0.4570


## CATBOOST
- all feats

In [50]:
# model_4_feats = ['alcohol',
#                 'volatile_acidity',
#                 'density',
#                 'wine_color',
#                 'chlorides',
#                 'sulphates',
#                 'citric_acid']
# X_train_1 = X_train[model_4_feats]
# X_validate_1 = X_validate[model_4_feats]
# X_test_1 = X_test[model_4_feats]

# Create and fit the thing
CATb = CatBoostRegressor(verbose=False,depth=10)
CATb.fit(X_train_1,y_train.quality)
y_train['target_pred_CATb'] = CATb.predict(X_train_1)
CATb.predict(X_train_1)
#Evaluate: RMSE
rmse_train = mean_squared_error(y_train.quality, y_train.target_pred_CATb)**(1/2)
#REPEAT STEPS 2-3
# predict validate
y_validate['target_pred_CATb'] = CATb.predict(X_validate_1)
# evaluate: RMSE
rmse_validate = mean_squared_error(y_validate.quality, y_validate.target_pred_CATb)**(1/2)
r2 = r2_score(y_train.quality,y_train.target_pred_CATb)
print(f"""RMSE for OLS using LinearRegression
Training/In-Sample:  {rmse_train} 
Validation/Out-of-Sample: {rmse_validate}
Train R2 Score: {r2}""")
#Append this to the metric_df
metric_df = metric_df.append(
    {'model': 'CATb maxD=10 ALL FEATS',
     'RMSE_train': round(rmse_train,2),
     'RMSE_validate': round(rmse_validate,2),
     'R2_validate': round(explained_variance_score(y_validate.quality,
                                             y_validate.target_pred_CATb),4)
    }, ignore_index=True)

metric_df

RMSE for OLS using LinearRegression
Training/In-Sample:  0.17022612101327936 
Validation/Out-of-Sample: 0.6172256509772919
Train R2 Score: 0.961386171715574


/var/folders/ty/2pnn6wsj06n_glzjgbmxzlkw0000gn/T/ipykernel_2015/3167343354.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metric_df = metric_df.append(


,model,RMSE_train,RMSE_validate,R2_validate
0,mean_baseline,0.90,0.90,0.0000
1,OLS Model 1 (base feats),0.74,0.76,0.2782
2,OLS Model 1 (base + color),0.74,0.75,0.2823
3,OLS Model 1 (+ chlorides),0.74,0.75,0.2836
4,OLS Model 1 (+ sulphates),0.74,0.75,0.2923
5,OLS Model 1 Scaled (+ sulphates),0.74,0.75,0.2923
6,DTR: maxD=5 (+ citric),0.70,0.75,0.2932
7,"RFR ALL FEATS est=301, maxD=10",0.45,0.66,0.4570
8,CATb maxD=10 ALL FEATS,0.17,0.62,0.5193


## CATBOOST
- test split

In [51]:
# predict on test split
y_test['target_pred_CATb'] = CATb.predict(X_test_1)
# evaluate: RMSE
rmse_test = mean_squared_error(y_test.quality, y_test.target_pred_CATb)**(1/2)
r2 = r2_score(y_test.quality,y_test.target_pred_CATb)
print(f"""RMSE for OLS using LinearRegression
Training/In-Sample:  {rmse_train} 
Validation/Out-of-Sample: {rmse_test}
Train R2 Score: {r2}""")

RMSE for OLS using LinearRegression
Training/In-Sample:  0.17022612101327936 
Validation/Out-of-Sample: 0.6133392651762423
Train R2 Score: 0.5060484108465191
